<a href="https://colab.research.google.com/github/Cherrymmu/AIFFEL_Online_Quest_Cherry/blob/main/%EB%A9%94%EC%9D%B8%ED%80%98%EC%8A%A4%ED%8A%B8/2%EB%B2%88(23_11_17).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> # 메인퀘스트 2번

아이펠 캠퍼스 학번, 퀘스트 점수 관리 시스템
Class Aiffel 만들기

- 이름
- 입학년도
- 기수
- 과정명

학번:7자리로 이루어짐

학번 생성 규칙:

- YY : 연도 마지막 두 자리 (ex. 2023년은 23)
- NN : 기수 (온라인 7기는 29기)
- C : 과정 종류 (리서치=1, 코어=2)
- XX : 가나다순 번호


https://python.flowdas.com/library/csv.html


In [265]:
import csv
import random

class Aiffel:
    id_counter = 0
    power = False  # 전원 상태 추가

    # Q1. 코어 명단 작성하기
    def __init__(self, 이름, 입학연도, 기수, 과정명):
        self.students = {}
        students_sorted = sorted(이름.keys())
        for index, name in enumerate(students_sorted):
            year = str(입학연도)[2:] # 연도 마지막 두자리
            number = str(기수).zfill(2) # 기수를 두자리 수로 zfill을 통해 맞춰줌
            course = str(과정명).zfill(1) # 과정 종류를 한자리 수로 맞춤
            student_number = year + number + course + str(index + 1).zfill(2) # 학생순서를의미: 문자열 길이 맞추기
            self.students[name] = {
                'id': student_number,
                'score': 0,
                'score_plus': 0,
                'penalty': 0,
                'guild_score': 0,
            }
            Aiffel.id_counter += 1

        self.power = True  # 전원 상태를 True로 설정

    # Q2. Aiffel 클래스에서 생성된 학번 출력 매서드
    @staticmethod
    def get_id_num():
        print(f"학번 개수는 {Aiffel.id_counter} 입니다.")

    def score(self, name, score_plus):
        if score_plus < 1 or score_plus > 3:
            print("점수를 부여해주세요")
        else:
            self.students[name]['score'] += score_plus
        self.students[name]['score_plus'] += score_plus
        print(f" {name} 학생에게 {score_plus}이 부여 되었습니다.")


    # Q4. 퀘스트 점수 차감
    def penalty(self, name, penalty):
        self.students[name]['score'] -= penalty
        self.students[name]['penalty'] += penalty
        print (f" {name} 학생에게 {penalty}이 차감 되었습니다.")

    # Q6. 중도포기자 입력
    def drop_out(self, name):
        try:
            del self.students[name]
            print(f"{name}을 명단에서 삭제하였습니다.")
        except KeyError:
            print(f"{name}은(는) 명단에 없습니다.")

    # Q7. 그룹은 총 8개 (상생, 신뢰, 열정, 이끔, 성장, 가치, 공유, 확산)
class Aiffel_Group(Aiffel):
    def __init__(self, 이름, 입학연도, 기수, 과정명):
        super().__init__(이름, 입학연도, 기수, 과정명)
        self.group_names = ["상생", "신뢰", "열정", "이끔", "성장", "가치", "공유", "확산"]
        self.groups = []


        # 'group_score' 항목 추가
        for student in self.students:
            self.students[student]['group_score'] = 0

    # Q7...
    def make_group(self, num):
      students = list(self.students.keys())
      random.shuffle(students)
      self.groups = [students[n:n+num] for n in range(0, len(students), num)]

      for i in range(len(self.groups)):
          for student in self.groups[i]:
              self.students[student]['group'] = self.group_names[i]

      for student, info in self.students.items():
          group_name = info['group']
          print(f"{student}의 그룹은 {group_name}입니다.")
      return self.groups

    #그룹 점수가 영향을 끼치는 것
    def add_group_score(self, group_name, group_score):
      group_count = 0
      for student, info in self.students.items():
          if 'group' in info and info['group'] == group_name:
              self.students[student]['guild_score'] += group_score
              self.students[student]['group_score'] += group_score
              group_count += 1
              print(f"{student} 학생의 그룹 점수가 {group_score}점 추가 되였습니다.")

  #  Q11
    def display_info(self):
        group_scores = {group_name: 0 for group_name in self.group_names}

        for student, info in self.students.items():
            if 'group' in info:
                group_scores[info['group']] += info['guild_score']

        with open('new_core_students.csv', 'w', newline='') as file:
            입력담당관 = csv.writer(file)
            입력담당관.writerow(["이름", "학번", "과정종류", "퀘스트 점수 총점", "퀘스트 입력 총점", "퀘스트 penalty 총합", "그룹명", "길드명", "길드 점수", "그룹별 스코어 총합"])
            for group_name, students in zip(self.group_names, self.groups):
                for student in students:
                    info = self.students[student]
                    입력담당관.writerow([student, info['id'], '코어', info['score'], info['score_plus'], info['penalty'], info['group'], '', info['guild_score'], group_scores.get(info['group'], '')])
        print(f"학생 정보를 등록하였습니다.")
# Q8
class Aiffel_Guild(Aiffel_Group):
    def __init__(self, 이름, 입학연도, 기수, 과정명):
        super().__init__(이름, 입학연도, 기수, 과정명)
        self.guilds = {}
        for student, info in self.students.items():
            info['group_score'] = 0
            info['guild_score'] = 0

    # 길드 추가
    def add_guild(self, guild_name):
        if guild_name not in self.guilds:
            self.guilds[guild_name] = []


    def add_student_to_guild(self, name, guild_name):
        if guild_name in self.guilds:
            if name in self.students:
                self.guilds[guild_name].append(name)
                self.students[name]['guild'] = guild_name
            else:
                print(f" {name} 학생이 학생 리스트에 없습니다.")
        else:
            print(f"{guild_name} 길드가 길드 리스트에 없습니다.")


    # 길드로 셔플해서 랜덤하게
    def make_guild(self):
        guilds_list = ["힌튼", "응", "하사비스"]
        students = list(self.students.keys())
        random.shuffle(students)
        self.guilds = {guild: [] for guild in guilds_list}
        for i in range(len(students)):
            guild = guilds_list[i % len(guilds_list)]
            self.guilds[guild].append(students[i])
            self.students[students[i]]['guild'] = guild

        for guild, students in self.guilds.items():
            print(f"{guild} 길드: {students}")
        return self.guilds

    # Q9 길드 점수 부여
    def add_guild_score(self, guild_name, score):
        if guild_name in self.guilds:
          for student in self.guilds[guild_name]:
              self.students[student]['guild_score'] += score
              print(f"{student} 학생의 길드 점수가 {score}점 증가하였습니다.")
        else:
          print(f"{guild_name} 길드가 존재하지 않습니다.")

    def add_group_score(self, group_name, group_score):
        for student, info in self.students.items():
            if 'group' in info and info['group'] == group_name:
                self.students[student]['guild_score'] += group_score

    def show_student_score(self, name):
        print(f" {name} 학생의 개인 점수: {self.students[name]['score']}")
        print(f" {name} 학생의 그룹 점수: {self.students[name]['group_score']}")
        print(f" {name} 학생의 길드 점수: {self.students[name]['guild_score']}")


    # Q11
    def display_info(self):
        group_scores = {group_name: 0 for group_name in self.group_names}
        guild_scores = {guild_name: 0 for guild_name in self.guilds}

        for student, info in self.students.items():
            if 'group' in info:
                group_scores[info['group']] += info['guild_score']
                guild_name = info.get('guild')
                if guild_name:
                    guild_scores[guild_name] += info['guild_score']

        with open('new_core_students.csv', 'w', newline='') as file:
          writer = csv.writer(file)
          writer.writerow(["이름", "학번", "과정종류", "퀘스트 점수 총점", "퀘스트 입력 총점", "퀘스트 penalty 총합", "그룹명", "길드명", "길드 점수", "그룹 점수", "그룹별 스코어 총합", "길드별 스코어 총합"])
          for group_name, students in zip(self.group_names, self.groups):
              for student in students:
                  info = self.students[student]
                  writer.writerow([student, info['id'], '코어', info['score'], info['score_plus'], info['penalty'], info['group'], info.get('guild', ''), info['guild_score'], info['group_score'], group_scores.get(info['group'], ''), guild_scores.get(info.get('guild', ''), '')])
        print(f"학생 정보를 등록하였습니다.")

#코어 명단
core_students = core_students = {
    '강영현': {'id': '학번1', '길드': '길드1'},
    '김경민': {'id': '학번2', '길드': '길드2'},
    '김민기': {'id': '학번3', '길드': '길드1'},
    '김승순': {'id': '학번4', '길드': '길드2'},
    '김양희': {'id': '학번5', '길드': '길드1'},
    '김영신': {'id': '학번6', '길드': '길드2'},
    '오우진': {'id': '학번7', '길드': '길드1'},
    '윤혜윤': {'id': '학번8', '길드': '길드2'},
    '이상원': {'id': '학번9', '길드': '길드1'},
    '이선재': {'id': '학번10', '길드': '길드2'},
    '이슬': {'id': '학번11', '길드': '길드1'},
    '이승환': {'id': '학번12', '길드': '길드2'},
    '전민규': {'id': '학번13', '길드': '길드1'},
    '조보겸': {'id': '학번14', '길드': '길드2'},
    '조수민': {'id': '학번15', '길드': '길드1'},
    '최강훈': {'id': '학번16', '길드': '길드2'},
    '한현종': {'id': '학번17', '길드': '길드1'},
    '홍채림': {'id': '학번18', '길드': '길드2'}
}
print(core_students)

입학연도 = 2023  # 입학연도
기수 = 29  # 기수
과정명 = 2 # 과정종류


{'강영현': {'id': '학번1', '길드': '길드1'}, '김경민': {'id': '학번2', '길드': '길드2'}, '김민기': {'id': '학번3', '길드': '길드1'}, '김승순': {'id': '학번4', '길드': '길드2'}, '김양희': {'id': '학번5', '길드': '길드1'}, '김영신': {'id': '학번6', '길드': '길드2'}, '오우진': {'id': '학번7', '길드': '길드1'}, '윤혜윤': {'id': '학번8', '길드': '길드2'}, '이상원': {'id': '학번9', '길드': '길드1'}, '이선재': {'id': '학번10', '길드': '길드2'}, '이슬': {'id': '학번11', '길드': '길드1'}, '이승환': {'id': '학번12', '길드': '길드2'}, '전민규': {'id': '학번13', '길드': '길드1'}, '조보겸': {'id': '학번14', '길드': '길드2'}, '조수민': {'id': '학번15', '길드': '길드1'}, '최강훈': {'id': '학번16', '길드': '길드2'}, '한현종': {'id': '학번17', '길드': '길드1'}, '홍채림': {'id': '학번18', '길드': '길드2'}}


In [266]:
# Aiffel_Group 객체
aiffel_group = Aiffel_Group(core_students, 2023, 29, 'core')

# 그룹 만들기
aiffel_group.make_group(6)  # 6명씩 그룹으로 묶기

# 그룹에 점수 부여하기
aiffel_group.add_group_score('열정', 2)

강영현의 그룹은 신뢰입니다.
김경민의 그룹은 상생입니다.
김민기의 그룹은 신뢰입니다.
김승순의 그룹은 상생입니다.
김양희의 그룹은 열정입니다.
김영신의 그룹은 상생입니다.
오우진의 그룹은 열정입니다.
윤혜윤의 그룹은 신뢰입니다.
이상원의 그룹은 신뢰입니다.
이선재의 그룹은 열정입니다.
이슬의 그룹은 상생입니다.
이승환의 그룹은 신뢰입니다.
전민규의 그룹은 신뢰입니다.
조보겸의 그룹은 열정입니다.
조수민의 그룹은 상생입니다.
최강훈의 그룹은 열정입니다.
한현종의 그룹은 상생입니다.
홍채림의 그룹은 열정입니다.
김양희 학생의 그룹 점수가 2점 추가 되였습니다.
오우진 학생의 그룹 점수가 2점 추가 되였습니다.
이선재 학생의 그룹 점수가 2점 추가 되였습니다.
조보겸 학생의 그룹 점수가 2점 추가 되였습니다.
최강훈 학생의 그룹 점수가 2점 추가 되였습니다.
홍채림 학생의 그룹 점수가 2점 추가 되였습니다.


In [259]:
aiffel_guild.add_guild_score('하사비스', 3)

윤혜윤 학생의 길드 점수가 3점 증가하였습니다.
오우진 학생의 길드 점수가 3점 증가하였습니다.
이선재 학생의 길드 점수가 3점 증가하였습니다.
이승환 학생의 길드 점수가 3점 증가하였습니다.
김경민 학생의 길드 점수가 3점 증가하였습니다.
이슬 학생의 길드 점수가 3점 증가하였습니다.


In [238]:
# 랜덤하게 길드 생성
aiffel_guild = Aiffel_Guild(core_students, 2023, 29, 'core')
guilds = aiffel_guild.make_guild()
aiffel_guild.display_info()

힌튼 길드: ['이상원', '홍채림', '김양희', '한현종', '조수민', '강영현']
응 길드: ['조보겸', '김민기', '최강훈', '김영신', '김승순', '전민규']
하사비스 길드: ['윤혜윤', '오우진', '이선재', '이승환', '김경민', '이슬']
학생 정보를 등록하였습니다.


In [240]:
# 개인점수, 그룹점수, 길드 점수
aiffel_guild.show_student_score('강영현')

 강영현 학생의 개인 점수: 0
 강영현 학생의 그룹 점수: 0
 강영현 학생의 길드 점수: 0


In [241]:
# 학번 개수 만들기
aiffel_instance = Aiffel(core_students, 입학연도, 기수, 과정명)
aiffel_instance.get_id_num()

학번 개수는 54 입니다.


In [228]:
# 학번
student_id = aiffel_instance.students['김승순']['id']
student_id

'2329204'

In [204]:
student_id = aiffel_instance.students['조수민']['id']
student_id

'2329215'

In [184]:
student_id = aiffel_instance.students['이슬']['id']
student_id

'2329211'

In [242]:
# 그루 점수 부여
aiffel_grew = Aiffel_Guild(core_students, 2023, 29, 'core')
aiffel_grew.score('홍채림', 2)
aiffel_grew.penalty('홍채림', 1)

 홍채림 학생에게 2이 부여 되었습니다.
 홍채림 학생에게 1이 차감 되었습니다.


In [267]:
aiffel_grew = Aiffel(core_students, 2023, 29, 'core')
aiffel_grew.drop_out('김경민')
aiffel_grew.drop_out('김영신')

김경민을 명단에서 삭제하였습니다.
김영신을 명단에서 삭제하였습니다.


# **회고**

메인 퀘스트 1번을 푸는데 너무 시간을 많이 쏟아서 제대로 실행해보지도 못하고 메인퀘스트 2번을 만들고 그냥 제출했다. 그래서 구글링한 내용 마구잡이로 집어넣다보니 오류가 계속 떴다.
특히 개인점수 그룹점수 길드점수를 나누어서 각각 점수를 더하고 빼주는 것이 쉽지 않았고 오늘 배운 판다스의 데이터 프레임을 활용했으면 조금 더 가독성이 좋은 출력값이 나오게 되었을 것 같아 아쉬웠다.

그래서 결론은
종종 이 코드를 찾아서 혼자 연구를 좀 더 진행해봐야겠다.

언젠가는 조금 더 가독성이 좋고, 가볍게 코드를 짤 수 있지 않을까한다.
